In [81]:
import csv
from itertools import islice, chain, compress
from datetime import datetime
from collections import namedtuple



In [26]:
def read_file(file_name):
    with open(file_name) as f:
        rows = csv.reader(f, delimiter=',', quotechar='"')
        for row in rows:
            yield cast_data_row(row)


def cast_value(value):
    def my_cast():
        yield datetime, lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ")
        yield int, lambda x: int(x)
        yield str, lambda x: str(x)

    for expect_type, cast_fn in my_cast():
        try:
            return cast_fn(value)
        except ValueError:
            continue


def cast_data_row(data_row):
    return [cast_value(v) for v in data_row]


pers_inf = read_file('update_status.csv')
for i in islice(pers_inf, 3):
    print(i)

['ssn', 'last_updated', 'created']


In [35]:
def create_obj_from_file(file_name):
    data = read_file(file_name)
    obj = namedtuple(file_name.removesuffix('.csv').capitalize(), *islice(data, 1))
    for row in data:
        yield obj(*row)


employment = create_obj_from_file('employment.csv')
pers_inf = create_obj_from_file('personal_info.csv')
upd_status = create_obj_from_file('update_status.csv')
vehicle = create_obj_from_file('vehicles.csv')

for i in islice(employment, 1):
    print(i)
for i in islice(pers_inf, 1):
    print(i)
for i in islice(upd_status, 1):
    print(i)
for i in islice(vehicle, 1):
    print(i)

Employment(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824')
Personal_info(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Update_status(ssn='100-53-9824', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30))
Vehicles(ssn='100-53-9824', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)


In [65]:
def create_obj_from_files(*file_names):
    headers = []
    for file in (read_file(f) for f in file_names):
        headers.append(*islice(file, 1))
    return headers

In [66]:
create_obj_from_files('employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv')

[['employer', 'department', 'employee_id', 'ssn'],
 ['ssn', 'first_name', 'last_name', 'gender', 'language'],
 ['ssn', 'last_updated', 'created'],
 ['ssn', 'vehicle_make', 'vehicle_model', 'model_year']]

In [82]:
def create_obj_from_files(*file_names):
    headers = chain(islice(file, 1) for file in (read_file(f) for f in file_names))
    return compress(headers, )

In [78]:
t = create_obj_from_files('employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv')
for i in t:
    for j in i:
        print(j)

['employer', 'department', 'employee_id', 'ssn']
['ssn', 'first_name', 'last_name', 'gender', 'language']
['ssn', 'last_updated', 'created']
['ssn', 'vehicle_make', 'vehicle_model', 'model_year']


In [83]:
t = create_obj_from_files('employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv')

TypeError: compress() missing required argument 'selectors' (pos 2)

In [130]:
def create_obj_from_files(*file_names):
    headers = [next(read_file(f)) for f in file_names]
    column_names = headers[0] + [col for header in headers[1:] for col in header[1:]]
    CombinedRow = namedtuple('CombinedRow', column_names)
    for rows in zip(*(read_file(f) for f in file_names)):
        combined_row = [rows[0][0]] + [col for row in rows for col in row[1:]]
        yield CombinedRow(*combined_row)



In [140]:
def create_obj_from_files(*file_names):
    headers = [next(read_file(f)) for f in file_names]
    column_names = headers[0] + [col for header in headers[1:] for col in header[1:]]
    CombinedRow = namedtuple('CombinedRow', column_names)
    for rows in zip(*(islice(read_file(f), 1, None) for f in file_names)):
        combined_row = [rows[0][0]] + list(chain.from_iterable(row[1:] for row in rows))
        yield CombinedRow(*combined_row)

In [92]:
create_obj_from_files('employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv')

[['employer', 'department', 'employee_id', 'ssn'],
 ['ssn', 'first_name', 'last_name', 'gender', 'language'],
 ['ssn', 'last_updated', 'created'],
 ['ssn', 'vehicle_make', 'vehicle_model', 'model_year']]

In [94]:
create_obj_from_files('employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv')

['employer',
 'department',
 'employee_id',
 'ssn',
 'first_name',
 'last_name',
 'gender',
 'language',
 'last_updated',
 'created',
 'vehicle_make',
 'vehicle_model',
 'model_year']

In [139]:
comb_obj = create_obj_from_files('employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv')
for i in islice(comb_obj, 2):
    print(i)

CombinedRow(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
CombinedRow(employer='Nicolas and Sons', department='Sales', employee_id='41-6841359', ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao', last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57), vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)


In [141]:
comb_obj = create_obj_from_files('employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv')
for i in islice(comb_obj, 2):
    print(i)

CombinedRow(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
CombinedRow(employer='Nicolas and Sons', department='Sales', employee_id='41-6841359', ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao', last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57), vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
